In [18]:
import random
import pandas as pd
import numpy as np
import math
import scipy.stats as stats
import matplotlib.pyplot as plt

class Node:
    def __init__(self, node_id, gate=None, Q=None, lambda_value=None, Ti=None, Tm=None, t=None, muu=None, std_dev=None):
        self.node_id = node_id
        self.gate = gate
        self.Q = Q
        self.lambda_value = lambda_value
        self.Ti = Ti
        self.Tm = Tm
        self.t = t
        self.muu = muu
        self.std_dev = std_dev
        self.dependencies = []
        self.status = False



def calculate_system_reliability(root_node):
    if root_node.status:
        return 1.0
    elif not root_node.gate:
        return calculate_Q(root_node)
    else:
        is_and_gate = root_node.gate.upper() == 'AND'
        if is_and_gate:
            reliability = 1.0
            for dep in root_node.dependencies:
                reliability *= calculate_system_reliability(node_map[dep])
            return reliability
        else:
            reliability = 0.0  # Initialize with 0 for OR gate
            for dep in root_node.dependencies:
                reliability += calculate_system_reliability(node_map[dep])  # Sum probabilities for OR gate
            return min(1, reliability)  # Ensure reliability is in [0, 1]



def calculate_Q(node):
    if node.lambda_value is not None and node.Ti is not None:
        return (1 - (1 / (node.lambda_value * node.Ti)) * (1 - np.exp(-node.lambda_value * node.Ti)))
    elif node.lambda_value is not None and node.Tm is not None:
        return 1 - np.exp(-node.lambda_value * node.Tm)
    elif node.lambda_value is not None and node.t is not None and node.muu is not None:
        return (node.lambda_value / (node.lambda_value + node.muu)) * (1 - np.exp(-(node.lambda_value + node.muu) * node.t))
    elif node.Q is not None:
        return node.Q
    else:
        return None

def get_cutset(node):
    if not node.gate:
        return [node.node_id]
    cutset = []
    for dep in node.dependencies:
        if node_map[dep].status:
            cutset.extend(get_cutset(node_map[dep]))
    return cutset

# ...

def build_fault_tree_from_excel(file_path):
    nodes = {}
    data = pd.read_excel(file_path, header=None)

    for row in data.iterrows():
        row_data = row[1]
        node_id = str(row_data[0])
        gate = str(row_data[1])
        Q = float(row_data[2]) if pd.notna(row_data[2]) else None
        lambda_value = float(row_data[3]) if pd.notna(row_data[3]) else None
        Ti = float(row_data[4]) if pd.notna(row_data[4]) else None
        Tm = float(row_data[5]) if pd.notna(row_data[5]) else None
        t = float(row_data[6]) if pd.notna(row_data[6]) else None
        muu = float(row_data[7]) if pd.notna(row_data[7]) else None
        std_dev = float(row_data[8]) if pd.notna(row_data[8]) else None
        dependencies = [str(dep) for dep in row_data[9:] if pd.notna(dep)]

        if gate and len(dependencies) > 0:
            node = Node(node_id, gate, std_dev=std_dev)  # Assign std_dev to all nodes
            node.dependencies = dependencies
            nodes[node_id] = node
        elif Q is not None:
            node = Node(node_id, Q=Q, std_dev=std_dev)  # Assign std_dev to all nodes
            nodes[node_id] = node
        elif lambda_value is not None:
            node = Node(node_id, lambda_value=lambda_value, Ti=Ti, Tm=Tm, t=t, muu=muu, std_dev=std_dev)  # Assign std_dev to all nodes
            nodes[node_id] = node

    return nodes

# ...


def print_fault_tree(node, indent="", is_last=True):
    if node.gate:
        connector = "└─" if is_last else "├─"
        print(indent + f"{connector} Node {node.node_id} ({node.gate}, Q={calculate_Q(node)}, Std Dev={node.std_dev})")
        for i, dep in enumerate(node.dependencies):
            is_last_child = i == len(node.dependencies) - 1
            print_fault_tree(node_map[dep], indent + ("   " if is_last else "│  "), is_last_child)
    else:
        connector = "└─" if is_last else "├─"
        print(indent + f"{connector} Node {node.node_id} (Q={calculate_Q(node)}, Std Dev={node.std_dev})")

def reset_node_statuses(node):
    node.status = False
    for dep in node.dependencies:
        reset_node_statuses(node_map[dep])

def monte_carlo_simulation(root_node, num_iterations):
    system_statuses = []
    cutsets = []
    encountered_cutsets = set()
    included_nodes = set()
    for _ in range(num_iterations):
        reset_node_statuses(root_node)
        randomize_dependencies(root_node)
        simulate_event(root_node)
        system_status = calculate_system_reliability(root_node)
        system_statuses.append(system_status)
        if system_status > 0:
            cutset = sorted(set(get_cutset(root_node)) - included_nodes)
            if tuple(cutset) not in encountered_cutsets and len(cutset) > 0:
                cutsets.append(cutset)
                encountered_cutsets.add(tuple(cutset))
                included_nodes.update(cutset)
    return cutsets, system_statuses

def randomize_dependencies(node):
    if not node.gate:
        return
    shuffled_dependencies = list(node.dependencies)
    random.shuffle(shuffled_dependencies)
    node.dependencies = shuffled_dependencies
    for dep in node.dependencies:
        randomize_dependencies(node_map[dep])

def simulate_event(node):
    if node.status:
        return
    if not node.gate:
        q_value = calculate_Q(node)
        if q_value is not None and random.random() <= q_value:
            node.status = True
    else:
        is_and_gate = node.gate.upper() == 'AND'
        if is_and_gate:
            for dep in node.dependencies:
                simulate_event(node_map[dep])
                if not node_map[dep].status:
                    return
            node.status = True
        else:
            for dep in node.dependencies:
                if not node_map[dep].status:
                    simulate_event(node_map[dep])
                if node_map[dep].status:
                    node.status = True
                    return

def calculate_system_reliability_for_cutsets(cutsets):
    total_reliability = 0.0
    for cutset in cutsets:
        reliability = 1.0
        for node_id in cutset:
            node = node_map[node_id]
            reliability *= calculate_system_reliability(node)
        total_reliability += reliability
    return total_reliability

def perform_uncertainty_analysis(cutsets):
    q_values = []
    for cutset in cutsets:
        q_value = 1.0
        for node_id in cutset:
            node = node_map[node_id]
            if node.std_dev is not None and node.Q is not None:
                q_value *= np.random.normal(loc=node.Q, scale=node.std_dev)
            elif node.Q is not None:
                q_value *= node.Q
            # Handle the case where node.Q is None (bypass uncertainty analysis)
            # else:
            #     q_value *= 1.0  # You can choose an appropriate default value here if needed
        q_values.append(q_value)
    return q_values


def plot_normal_distribution(q_values):
    mean = np.mean(q_values)
    std_dev = np.std(q_values)
    min_val = np.min(q_values)
    max_val = np.max(q_values)
    percentile_5 = np.percentile(q_values, 5)
    percentile_95 = np.percentile(q_values, 95)

    plt.figure(figsize=(10, 6))
    plt.title("Normal Distribution for Estimated System Failure Probability")
    plt.xlabel("Estimated System Failure Probability (Q)")
    plt.ylabel("Probability Density")

    x = np.linspace(min_val, max_val, 1000)
    y = stats.norm.pdf(x, mean, std_dev)

    plt.plot(x, y, label="Normal Distribution")
    plt.axvline(x=mean, color='r', linestyle='--', label=f"Mean: {mean:.4f}")
    plt.axvline(x=percentile_5, color='g', linestyle='--', label=f"5th Percentile: {percentile_5:.4f}")
    plt.axvline(x=percentile_95, color='b', linestyle='--', label=f"95th Percentile: {percentile_95:.4f}")

    plt.legend()
    plt.grid()
    plt.show()



# ...

if __name__ == "__main__":
    excel_file_path = "/content/lop.xlsx"  # Update with your Excel file path
    node_map = build_fault_tree_from_excel(excel_file_path)

    root_id = list(node_map.keys())[0]

    print("\nGenerated Fault Tree:")
    if root_id in node_map:
        print_fault_tree(node_map[root_id])

        # Check if any standard deviation values are present
        standard_deviation_present = any(node.std_dev is not None for node in node_map.values())

        num_iterations = 1000000
        cutsets, _ = monte_carlo_simulation(node_map[root_id], num_iterations)

        print("\nEstimated Minimal Cutsets:")
        for i, cutset in enumerate(cutsets, start=1):
            print(f"Cutset {i}: {'  '.join(cutset)}")

        # Perform uncertainty analysis only if standard deviation values are present
        if standard_deviation_present:
            q_values = perform_uncertainty_analysis(cutsets)

            # Calculate mean as the sum of Estimated System Failure Probabilities for all cutsets (OR gates)
            mean_q = np.sum([calculate_system_reliability_for_cutsets([cutset]) for cutset in cutsets])

            # Calculate standard deviation as the square root of the sum of squared standard deviations
            std_dev_q = np.sqrt(np.sum([node_map[node_id].std_dev ** 2 for cutset in cutsets for node_id in cutset]))

            print("\nEstimated System Failure Probability (with Uncertainty Analysis):")
            print(f"Mean: {mean_q:.5e}")
            print(f"Standard Deviation: {std_dev_q:.4f}")

            # Calculate max, min, 5th, and 95th percentiles for the normal distribution
            max_q = mean_q + (std_dev_q * stats.norm.ppf(0.95))
            min_q = mean_q - (std_dev_q * stats.norm.ppf(0.95))
            percentile_5_q = stats.norm.ppf(0.05, loc=mean_q, scale=std_dev_q)
            percentile_95_q = stats.norm.ppf(0.95, loc=mean_q, scale=std_dev_q)

            print("\nStatistics for Normal Distribution:")
            print(f"Maximum: {max_q:.4f}")
            print(f"Minimum: {min_q:.4f}")
            print(f"5th Percentile: {percentile_5_q:.4f}")
            print(f"95th Percentile: {percentile_95_q:.4f}")

            # Plot normal distribution
            plt.figure(figsize=(10, 6))
            plt.title("Normal Distribution for Estimated System Failure Probability (with Uncertainty Analysis)")
            plt.xlabel("Estimated System Failure Probability (Q)")
            plt.ylabel("Probability Density")

            x = np.linspace(min_q, max_q, 1000)
            y = stats.norm.pdf(x, mean_q, std_dev_q)

            plt.plot(x, y, label="Normal Distribution")
            plt.axvline(x=mean_q, color='r', linestyle='--', label=f"Mean: {mean_q:.4f}")
            plt.axvline(x=percentile_5_q, color='g', linestyle='--', label=f"5th Percentile: {percentile_5_q:.4f}")
            plt.axvline(x=percentile_95_q, color='b', linestyle='--', label=f"95th Percentile: {percentile_95_q:.4f}")

            plt.legend()
            plt.grid()
            plt.show()
        else:
            print("\nUncertainty analysis skipped: No standard deviation values provided.")

        # Calculate and print Estimated System Failure Probability without uncertainty analysis
        q_values_no_uncertainty = calculate_system_reliability_for_cutsets(cutsets)
        print("\nEstimated System Failure Probability (without Uncertainty Analysis):")
        print(f"Probability: {q_values_no_uncertainty:.5e}")


    else:
        print("Root node not found in the tree.")





Generated Fault Tree:
└─ Node ACC (or, Q=None, Std Dev=None)
   ├─ Node ACC1 (and, Q=None, Std Dev=None)
   │  ├─ Node ACC4 (or, Q=None, Std Dev=None)
   │  │  ├─ Node ALOCA (Q=0.0, Std Dev=None)
   │  │  ├─ Node ACC-MOV-PG-1865A (Q=0.00065, Std Dev=None)
   │  │  ├─ Node ACC-CKV-FT-CV107 (Q=0.0001, Std Dev=None)
   │  │  └─ Node ACC-CKV-FT-CV109 (Q=0.0001, Std Dev=None)
   │  └─ Node ACC5 (or, Q=None, Std Dev=None)
   │     ├─ Node ACC-CKV-FT-CV130 (Q=0.0001, Std Dev=None)
   │     ├─ Node ACC-MOV-PG-1865B (Q=0.00065, Std Dev=None)
   │     └─ Node ACC-CKV-FT-cv128 (Q=0.0001, Std Dev=None)
   ├─ Node ACC2 (and, Q=None, Std Dev=None)
   │  ├─ Node ACC4 (or, Q=None, Std Dev=None)
   │  │  ├─ Node ALOCA (Q=0.0, Std Dev=None)
   │  │  ├─ Node ACC-MOV-PG-1865A (Q=0.00065, Std Dev=None)
   │  │  ├─ Node ACC-CKV-FT-CV107 (Q=0.0001, Std Dev=None)
   │  │  └─ Node ACC-CKV-FT-CV109 (Q=0.0001, Std Dev=None)
   │  └─ Node ACC6 (or, Q=None, Std Dev=None)
   │     ├─ Node ACC-CKV-FT-CV147 (Q=0.000

In [19]:
def calculate_FV_importance(cutsets):
    FV_importance_values = []

    # Step 1: Calculate Fx (probability without uncertainty analysis)
    Fx = q_values_no_uncertainty

    for cutset in cutsets:
        F_i = 1.0
        for node_id in cutset:
            node = node_map[node_id]
            if node.Q is not None:
                F_i *= node.Q

        FV = F_i / Fx
        FV_importance_values.append(FV)

    return FV_importance_values


In [20]:
FV_importance_values = calculate_FV_importance(cutsets)

print("\nFussell-Vesely (FV) Importance for Minimal Cutsets:")
for i, FV_importance in enumerate(FV_importance_values, start=1):
    print(f"Cutset {i}: FV = {FV_importance:.4f}")


Fussell-Vesely (FV) Importance for Minimal Cutsets:
Cutset 1: FV = 0.0001
Cutset 2: FV = 0.9999


In [21]:
Fx = q_values_no_uncertainty


In [22]:

RRR_importance_values = []

for i, cutset in enumerate(cutsets, start=1):
    reset_node_statuses(node_map[root_id])  # Reset node statuses before each calculation

        # Calculate Fx for the current cutset
    F_cutset = calculate_system_reliability_for_cutsets([cutset])

        # Set one minimal cutset to zero by excluding it from the calculation
    other_cutsets = [c for j, c in enumerate(cutsets) if j != i - 1]
    F0_cutset = calculate_system_reliability_for_cutsets(other_cutsets)

        # Calculate RRR for the current cutset
    RRR = Fx / F0_cutset
    RRR_importance_values.append(RRR)

print("\nRisk Reduction Importance (RRR) for Minimal Cutsets:")
for i, RRR_importance in enumerate(RRR_importance_values, start=1):
    print(f"Cutset {i}: RRR = {RRR_importance:.4f}")





Risk Reduction Importance (RRR) for Minimal Cutsets:
Cutset 1: RRR = 1.0001
Cutset 2: RRR = 10001.0000


In [23]:

RRD_importance_values = []

for i, cutset in enumerate(cutsets, start=1):
    reset_node_statuses(node_map[root_id])  # Reset node statuses before each calculation

        # Calculate Fx for the current cutset
    F_cutset = calculate_system_reliability_for_cutsets([cutset])

        # Set one minimal cutset to zero by excluding it from the calculation
    other_cutsets = [c for j, c in enumerate(cutsets) if j != i - 1]
    F0_cutset = calculate_system_reliability_for_cutsets(other_cutsets)

    # Calculate RRD for the current cutset
    RRD = Fx - F0_cutset
    RRD_importance_values.append(RRD)

print("\nRisk Reduction Difference (RRD) for Minimal Cutsets:")
for i, RRD_importance in enumerate(RRD_importance_values, start=1):
    print(f"Cutset {i}: RRD = {RRD_importance:.4f}")





Risk Reduction Difference (RRD) for Minimal Cutsets:
Cutset 1: RRD = 0.0000
Cutset 2: RRD = 0.0006


In [24]:

RIR_importance_values = []

for cutset in cutsets:
    reset_node_statuses(node_map[root_id])  # Reset node statuses before each calculation

        # Calculate F1 by setting all nodes in the current cutset to "operational" (probability = 1)
    for node_id in cutset:
        node_map[node_id].status = True  # Set each node in the cutset to "operational" (probability = 1)

    F1_cutset = calculate_system_reliability(node_map[root_id])

        # Calculate RIR for the current cutset
    RIR = F1_cutset / Fx
    RIR_importance_values.append(RIR)

print("\nRisk Increase Ratio (RIR) for Minimal Cutsets:")
for i, RIR_importance in enumerate(RIR_importance_values, start=1):
    print(f"Cutset {i}: RIR = {RIR_importance:.4f}")





Risk Increase Ratio (RIR) for Minimal Cutsets:
Cutset 1: RIR = 1538.3077
Cutset 2: RIR = 2.6162


In [25]:
RID_importance_values = []

for cutset in cutsets:
    reset_node_statuses(node_map[root_id])  # Reset node statuses before each calculation
    for node_id in cutset:
        node_map[node_id].status = True  # Set each node in the cutset to "failed"
    F1_cutset = calculate_system_reliability(node_map[root_id])
    RID = F1_cutset - Fx  # Calculate RID for the current cutset
    RID_importance_values.append(RID)

print("\nRisk Increase Difference (RID) for Minimal Cutsets:")
for i, RID_importance in enumerate(RID_importance_values, start=1):
    print(f"Cutset {i}: RID = {RID_importance:.4f}")





Risk Increase Difference (RID) for Minimal Cutsets:
Cutset 1: RID = 0.9993
Cutset 2: RID = 0.0011


In [26]:

B_importance_values = []

for cutset in cutsets:
    reset_node_statuses(node_map[root_id])  # Reset node statuses before each calculation

        # Calculate F1 by setting all nodes in the current cutset to "operational" (probability = 1)
    for node_id in cutset:
        node_map[node_id].status = True  # Set each node in the cutset to "operational" (probability = 1)

    F1_cutset = calculate_system_reliability(node_map[root_id])

        # Reset node statuses again for the next calculation
    reset_node_statuses(node_map[root_id])

        # Set all nodes in the current cutset to "failed" (probability = 0)
    for node_id in cutset:
        node_map[node_id].status = False

        # Calculate F0
    F0_cutset = calculate_system_reliability(node_map[root_id])

    B = F1_cutset - F0_cutset  # Calculate B for the current cutset
    B_importance_values.append(B)

print("\nBirnbaum Importance (B) for Minimal Cutsets:")
for i, B_importance in enumerate(B_importance_values, start=1):
    print(f"Cutset {i}: B = {B_importance:.4f}")





Birnbaum Importance (B) for Minimal Cutsets:
Cutset 1: B = 1.0000
Cutset 2: B = 0.0017
